In [1]:
import ROOT
import numpy as np
import pandas as pd 
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter

ROOT.gROOT.ProcessLine( "gErrorIgnoreLevel = kError;");
ROOT.gStyle.SetCanvasDefW(2400);
ROOT.gStyle.SetCanvasDefH(900);
ROOT.gStyle.SetLegendTextSize(0.05)
ROOT.gStyle.SetLabelSize(0.05)
ROOT.gStyle.SetMarkerSize(1)
ROOT.gStyle.SetMarkerStyle(8)
ROOT.gStyle.SetLineWidth(2)
ROOT.gStyle.SetTickLength(0.02, "y")

Welcome to JupyROOT 6.31/01


In [2]:
def convert_to_type(df, col, type):
    df[col] =  df[col].apply(lambda s: type(s))

def normalize_df(df, cols, norm):
    for col in cols:
        df[col] = df[col].div(df[norm])

In [3]:
df = pd.read_csv("das6-gpu/20231127-085724")
df["nvals"] =  df["input"].apply(lambda s: np.float64(s.split("_")[-1].split(".")[0]))
df["distribution"] =  df["input"].apply(lambda s: s.split("_")[1])
convert_to_type(df, "edges", np.bool_)
convert_to_type(df, "ttotal", np.float64)
del df["input"]
df

,iter,env,gpu,nbins,bulksize,edges,tfindbin,tfill,tstats,ttotal,nvals,distribution
0,0,CUDA_HIST,A4000,1,32768,False,0.0,0.0,0.0,3.156292,5.000000e+07,uniform
1,0,CUDA_HIST,A4000,1,32768,True,0.0,0.0,0.0,3.150113,5.000000e+07,uniform
2,0,CUDA_HIST,A4000,1,32768,False,0.0,0.0,0.0,3.743976,1.000000e+08,uniform
3,0,CUDA_HIST,A4000,1,32768,True,0.0,0.0,0.0,3.594731,1.000000e+08,uniform
4,0,CUDA_HIST,A4000,1,32768,False,0.0,0.0,0.0,7.631987,5.000000e+08,uniform
...,...,...,...,...,...,...,...,...,...,...,...,...
955,4,SYCL_HIST,A6000,1000,32768,True,0.0,0.0,0.0,11.941205,1.000000e+08,uniform
956,4,SYCL_HIST,A6000,1000,32768,False,0.0,0.0,0.0,23.154996,5.000000e+08,uniform
957,4,SYCL_HIST,A6000,1000,32768,True,0.0,0.0,0.0,22.325353,5.000000e+08,uniform
958,4,SYCL_HIST,A6000,1000,32768,False,0.0,0.0,0.0,36.665283,1.000000e+09,uniform


In [4]:
gp = df.groupby(["env", "gpu", "distribution", "nvals", "nbins", "bulksize", "edges"])
gp.mean()

iter  \
env       gpu   distribution nvals        nbins bulksize edges         
CUDA_HIST A2    uniform      5.000000e+07 1     32768    False   2.0   
                                                         True    2.0   
                                          10    32768    False   2.0   
                                                         True    2.0   
                                          100   32768    False   2.0   
...                                                              ...   
SYCL_HIST A6000 uniform      1.000000e+09 10    32768    True    2.0   
                                          100   32768    False   2.0   
                                                         True    2.0   
                                          1000  32768    False   2.0   
                                                         True    2.0   

                                                                tfindbin  \
env       gpu   distribution nvals        nbins bulksize edges             
CUDA_HIST A2    uniform      5.000000e+07 1     32768    False       0.0   
                                                         True        0.0   
                                          10    32768    False       0.0   
                                                         True        0.0   
                                          100   32768    False       0.0   
...                                                                  ...   
SYCL_HIST A6000 uniform      1.000000e+09 10    32768    True        0.0   
                                          100   32768    False       0.0   
                                                         True        0.0   
                                          1000  32768    False       0.0   
                                                         True        0.0   

                                                                tfill  tstats  \
env       gpu   distribution nvals        nbins bulksize edges                  
CUDA_HIST A2    uniform      5.000000e+07 1     32768    False    0.0     0.0   
                                                         True     0.0     0.0   
                                          10    32768    False    0.0     0.0   
                                                         True     0.0     0.0   
                                          100   32768    False    0.0     0.0   
...                                                               ...     ...   
SYCL_HIST A6000 uniform      1.000000e+09 10    32768    True     0.0     0.0   
                                          100   32768    False    0.0     0.0   
                                                         True     0.0     0.0   
                                          1000  32768    False    0.0     0.0   
                                                         True     0.0     0.0   

                                                                   ttotal  
env       gpu   distribution nvals        nbins bulksize edges             
CUDA_HIST A2    uniform      5.000000e+07 1     32768    False   2.951207  
                                                         True    2.896822  
                                          10    32768    False   3.085684  
                                                         True    2.907051  
                                          100   32768    False   2.985212  
...                                                                   ...  
SYCL_HIST A6000 uniform      1.000000e+09 10    32768    True   36.561308  
                                          100   32768    False  36.470976  
                                                         True   36.272586  
                                          1000  32768    False  36.327778  
                                                         True   36.519984  

[192 rows x 5 columns]

In [5]:
unique_env = df["env"].unique()
unique_bulksize = df["bulksize"].unique()
unique_nbins = df["nbins"].unique()
unique_nvals = df["nvals"].unique()
unique_gpus = df["gpu"].unique()
unique_distributions = df["distribution"].unique()
unique_edges = [True, False]
unique_env, unique_gpus, unique_nbins, unique_bulksize, unique_edges, unique_nvals, unique_distributions

(array(['CUDA_HIST', 'SYCL_HIST'], dtype=object),
 array(['A4000', 'A2', 'A6000'], dtype=object),
 array([   1,   10,  100, 1000]),
 array([32768]),
 [True, False],
 array([5.e+07, 1.e+08, 5.e+08, 1.e+09]),
 array(['uniform'], dtype=object))

## A4000

In [6]:
selected_env = unique_env
selected_bulksize = 32768
selected_bins = 100
selected_nvals = unique_nvals
selected_gpu = "A4000"
selected_distr = "uniform"
selected_edges = True

In [12]:
%jsroot on
title = f"Total Runtime of Histo1D with Different Input Sizes on NVIDIA A4000"

w = 1600
h = 900
c = ROOT.TCanvas("c1", title, w, h)
c.SetRightMargin(0.33)
c.SetBottomMargin(0.3)

mg = ROOT.TMultiGraph()

l = ROOT.TLegend(0.687, 0.533, 0.99, 0.9)
l.SetTextSize(0.03)

ROOT.gStyle.SetPalette(ROOT.kRainbow)  
for ei, env in enumerate(selected_env):
    # avg = np.array([float(results[(env, selected_bulksize, selected_bins, p)].mean()["time"]) for p in selected_nvals])
    # std = np.array([float(results[(env, selected_bulksize, selected_bins, p)].std() ["time"]) for p in selected_nvals])
    avg = np.array(gp.mean().loc[env, selected_gpu, selected_distr, :, 
                   selected_bins, selected_bulksize, selected_edges]["ttotal"], dtype=np.float64)
    std = np.array(gp.std().loc[env, selected_gpu, selected_distr, :,
                   selected_bins, selected_bulksize, selected_edges]["ttotal"], dtype=np.float64)

    gr = ROOT.TGraphErrors(len(selected_nvals), selected_nvals.astype(np.float64), avg, 
                           np.repeat(0., len(selected_nvals)), std)
    # gr.SetTitle("TGraphErrors Example");
    color = ROOT.TColor.GetPalette()[50+ 60 * ei]
    gr.SetMarkerColor(color)
    gr.SetLineColor(color)
    gr.SetLineWidth(4)
    gr.SetMarkerSize(2)
    gr.SetMarkerStyle(21)
    
    gr.GetHistogram().SetMinimum(0)
    gr.GetHistogram().SetLineWidth(10)

    mg.Add(gr, "ALP")
    l.AddEntry(gr, env.split("_")[0])

mg.SetTitle(title)

xaxis = mg.GetXaxis()
# xaxis.SetTitle("#splitline{Bulk Size}")
xaxis.SetTitle("  Number of doubles")
# xaxis.SetRangeUser(0, 10000000)
# xaxis.SetTitleOffset(1.5)
xaxis.SetTitleSize(0.05)
xaxis.SetLabelSize(0.05)
# xaxis.LabelsOption("hM")
# xaxis.SetTickSize(0)

yaxis = mg.GetYaxis()
yaxis.SetTitle("Time (s)")
yaxis.SetTitleOffset(1)
yaxis.SetTitleSize(0.05)
yaxis.SetLabelSize(0.05)

l.SetTextSize(0.06)

ROOT.gStyle.SetTitleFontSize(0.1)

mg.Draw("a")
l.Draw()
c.Draw()
# c.SaveAs(f"nsys_histogram_{env}.png")

## A6000

## A2